# Практикум 4. Кластеризация данных о зернах пшеницы

## Задание 1. Загрузка и предобработка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Настройка для корректного отображения русских символов
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# Загрузка данных
data = pd.read_csv('seeds_dataset.txt', sep='\t', header=None)

# Названия признаков (согласно описанию набора данных)
feature_names = ['Area', 'Perimeter', 'Compactness', 'Length of kernel', 
                'Width of kernel', 'Asymmetry coefficient', 'Length of kernel groove']
data.columns = feature_names + ['Class']

print("Размер набора данных:", data.shape)
print("\nПервые 5 строк:")
print(data.head())
print("\nИнформация о данных:")
print(data.info())
print("\nОписательная статистика:")
print(data.describe())

In [ ]:
# Проверка на пропущенные значения
print("Пропущенные значения:")
print(data.isnull().sum())

# Проверка на дубликаты
print(f"\nКоличество дубликатов: {data.duplicated().sum()}")

# Удаление строк с пропущенными значениями (если есть)
data_clean = data.dropna()
print(f"\nРазмер данных после удаления пропущенных значений: {data_clean.shape}")

# Разделение на признаки и метки
X = data_clean[feature_names]
y = data_clean['Class']

print(f"\nКоличество признаков: {X.shape[1]}")
print(f"Тип признаков: все числовые (float64)")
print(f"Количество классов: {y.nunique()}")
print(f"Распределение классов:")
print(y.value_counts().sort_index())

In [ ]:
# Стандартизация числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=feature_names)

print("Данные после стандартизации:")
print(X_scaled_df.describe())

# Визуализация распределения признаков до и после стандартизации
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for i, feature in enumerate(feature_names):
    axes[i].hist(X[feature], alpha=0.7, label='Исходные данные', bins=20)
    axes[i].hist(X_scaled_df[feature], alpha=0.7, label='Стандартизированные', bins=20)
    axes[i].set_title(feature)
    axes[i].legend()

plt.tight_layout()
plt.show()

## Задание 2. Кластеризация K-Means с k=3

In [ ]:
# Применение K-Means с k=3
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Вычисление коэффициента силуэта
silhouette_avg = silhouette_score(X_scaled, clusters)
print(f"Коэффициент силуэта для k=3: {silhouette_avg:.3f}")

# Сравнение с истинными классами
print("\nСравнение кластеров с истинными классами:")
comparison = pd.DataFrame({'Истинный_класс': y, 'Предсказанный_кластер': clusters})
print(comparison.groupby(['Истинный_класс', 'Предсказанный_кластер']).size().unstack(fill_value=0))

In [ ]:
# Визуализация кластеров с помощью PCA для 2D представления
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

plt.figure(figsize=(12, 5))

# Истинные классы
plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.title('Истинные классы')
plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.2%} дисперсии)')
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.2%} дисперсии)')

# Предсказанные кластеры
plt.subplot(1, 2, 2)
scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=clusters, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.title('K-Means кластеры (k=3)')
plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.2%} дисперсии)')
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.2%} дисперсии)')

plt.tight_layout()
plt.show()

## Задание 3. PCA для уменьшения размерности

In [ ]:
# Эксперимент с различным количеством главных компонентов
n_components_range = range(2, 7)
pca_results = []

for n_components in n_components_range:
    # Применение PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # K-Means кластеризация
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_pca)
    
    # Вычисление коэффициента силуэта
    silhouette_avg = silhouette_score(X_pca, clusters)
    
    pca_results.append({
        'n_components': n_components,
        'explained_variance_ratio': pca.explained_variance_ratio_.sum(),
        'silhouette_score': silhouette_avg
    })
    
    print(f"n_components={n_components}: silhouette_score={silhouette_avg:.3f}, "
          f"explained_variance={pca.explained_variance_ratio_.sum():.3f}")

pca_df = pd.DataFrame(pca_results)
print("\nРезультаты PCA:")
print(pca_df)

In [ ]:
# Визуализация результатов PCA
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# График коэффициента силуэта
axes[0].plot(pca_df['n_components'], pca_df['silhouette_score'], 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Количество главных компонентов')
axes[0].set_ylabel('Коэффициент силуэта')
axes[0].set_title('Зависимость коэффициента силуэта от количества PC')
axes[0].grid(True, alpha=0.3)

# График объясненной дисперсии
axes[1].plot(pca_df['n_components'], pca_df['explained_variance_ratio'], 'ro-', linewidth=2, markersize=8)
axes[1].set_xlabel('Количество главных компонентов')
axes[1].set_ylabel('Объясненная дисперсия')
axes[1].set_title('Зависимость объясненной дисперсии от количества PC')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Определение оптимального количества компонентов
best_pca_n = pca_df.loc[pca_df['silhouette_score'].idxmax(), 'n_components']
print(f"\nОптимальное количество главных компонентов: {best_pca_n}")
print(f"Лучший коэффициент силуэта: {pca_df['silhouette_score'].max():.3f}")

In [ ]:
# Визуализация кластеров в пространстве первых двух главных компонентов
pca_best = PCA(n_components=2)
X_pca_best = pca_best.fit_transform(X_scaled)

kmeans_best = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters_best = kmeans_best.fit_predict(X_pca_best)

plt.figure(figsize=(12, 5))

# Истинные классы
plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca_best[:, 0], X_pca_best[:, 1], c=y, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.title('Истинные классы (PCA)')
plt.xlabel(f'PC1 ({pca_best.explained_variance_ratio_[0]:.2%} дисперсии)')
plt.ylabel(f'PC2 ({pca_best.explained_variance_ratio_[1]:.2%} дисперсии)')

# K-Means кластеры
plt.subplot(1, 2, 2)
scatter = plt.scatter(X_pca_best[:, 0], X_pca_best[:, 1], c=clusters_best, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.title('K-Means кластеры (PCA)')
plt.xlabel(f'PC1 ({pca_best.explained_variance_ratio_[0]:.2%} дисперсии)')
plt.ylabel(f'PC2 ({pca_best.explained_variance_ratio_[1]:.2%} дисперсии)')

plt.tight_layout()
plt.show()

silhouette_pca = silhouette_score(X_pca_best, clusters_best)
print(f"Коэффициент силуэта для PCA (2 компонента): {silhouette_pca:.3f}")

## Задание 4. t-SNE для уменьшения размерности

In [ ]:
# Эксперимент с различным количеством компонентов t-SNE
n_components_range = range(2, 7)
tsne_results = []

for n_components in n_components_range:
    # Применение t-SNE
    tsne = TSNE(n_components=n_components, random_state=42, perplexity=30)
    X_tsne = tsne.fit_transform(X_scaled)
    
    # K-Means кластеризация
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_tsne)
    
    # Вычисление коэффициента силуэта
    silhouette_avg = silhouette_score(X_tsne, clusters)
    
    tsne_results.append({
        'n_components': n_components,
        'silhouette_score': silhouette_avg
    })
    
    print(f"n_components={n_components}: silhouette_score={silhouette_avg:.3f}")

tsne_df = pd.DataFrame(tsne_results)
print("\nРезультаты t-SNE:")
print(tsne_df)

In [ ]:
# Визуализация результатов t-SNE
plt.figure(figsize=(10, 6))
plt.plot(tsne_df['n_components'], tsne_df['silhouette_score'], 'go-', linewidth=2, markersize=8)
plt.xlabel('Количество компонентов t-SNE')
plt.ylabel('Коэффициент силуэта')
plt.title('Зависимость коэффициента силуэта от количества компонентов t-SNE')
plt.grid(True, alpha=0.3)
plt.show()

# Определение оптимального количества компонентов
best_tsne_n = tsne_df.loc[tsne_df['silhouette_score'].idxmax(), 'n_components']
print(f"\nОптимальное количество компонентов t-SNE: {best_tsne_n}")
print(f"Лучший коэффициент силуэта: {tsne_df['silhouette_score'].max():.3f}")

In [ ]:
# Визуализация кластеров в пространстве первых двух компонентов t-SNE
tsne_2d = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne_2d = tsne_2d.fit_transform(X_scaled)

kmeans_tsne = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters_tsne = kmeans_tsne.fit_predict(X_tsne_2d)

plt.figure(figsize=(12, 5))

# Истинные классы
plt.subplot(1, 2, 1)
scatter = plt.scatter(X_tsne_2d[:, 0], X_tsne_2d[:, 1], c=y, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.title('Истинные классы (t-SNE)')
plt.xlabel('t-SNE компонент 1')
plt.ylabel('t-SNE компонент 2')

# K-Means кластеры
plt.subplot(1, 2, 2)
scatter = plt.scatter(X_tsne_2d[:, 0], X_tsne_2d[:, 1], c=clusters_tsne, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.title('K-Means кластеры (t-SNE)')
plt.xlabel('t-SNE компонент 1')
plt.ylabel('t-SNE компонент 2')

plt.tight_layout()
plt.show()

silhouette_tsne = silhouette_score(X_tsne_2d, clusters_tsne)
print(f"Коэффициент силуэта для t-SNE (2 компонента): {silhouette_tsne:.3f}")

## Задание 5. Исследование влияния инициализации центроидов

In [ ]:
# Исследование различных методов инициализации
init_methods = ['k-means++', 'random']
random_states = [0, 42, 100, 200, 500]

init_results = []

for init_method in init_methods:
    for random_state in random_states:
        kmeans = KMeans(n_clusters=3, init=init_method, random_state=random_state, n_init=10)
        clusters = kmeans.fit_predict(X_scaled)
        
        silhouette_avg = silhouette_score(X_scaled, clusters)
        inertia = kmeans.inertia_
        
        init_results.append({
            'init_method': init_method,
            'random_state': random_state,
            'silhouette_score': silhouette_avg,
            'inertia': inertia
        })
        
        print(f"{init_method}, random_state={random_state}: silhouette={silhouette_avg:.3f}, inertia={inertia:.2f}")

init_df = pd.DataFrame(init_results)
print("\nРезультаты исследования инициализации:")
print(init_df.groupby('init_method').agg({
    'silhouette_score': ['mean', 'std', 'min', 'max'],
    'inertia': ['mean', 'std', 'min', 'max']
}).round(3))

In [ ]:
# Визуализация результатов инициализации
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# График коэффициента силуэта
for method in init_methods:
    method_data = init_df[init_df['init_method'] == method]
    axes[0].plot(method_data['random_state'], method_data['silhouette_score'], 
                'o-', label=method, linewidth=2, markersize=6)

axes[0].set_xlabel('Random State')
axes[0].set_ylabel('Коэффициент силуэта')
axes[0].set_title('Влияние инициализации на коэффициент силуэта')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# График инерции
for method in init_methods:
    method_data = init_df[init_df['init_method'] == method]
    axes[1].plot(method_data['random_state'], method_data['inertia'], 
                'o-', label=method, linewidth=2, markersize=6)

axes[1].set_xlabel('Random State')
axes[1].set_ylabel('Инерция')
axes[1].set_title('Влияние инициализации на инерцию')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Статистический анализ
print("\nСтатистический анализ методов инициализации:")
print("\nКоэффициент силуэта:")
for method in init_methods:
    method_scores = init_df[init_df['init_method'] == method]['silhouette_score']
    print(f"{method}: mean={method_scores.mean():.3f}, std={method_scores.std():.3f}")

print("\nИнерция:")
for method in init_methods:
    method_inertia = init_df[init_df['init_method'] == method]['inertia']
    print(f"{method}: mean={method_inertia.mean():.2f}, std={method_inertia.std():.2f}")

## Задание 6. Определение оптимального значения k

In [ ]:
# Эксперимент с различным количеством кластеров
k_range = range(2, 11)
k_results = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    silhouette_avg = silhouette_score(X_scaled, clusters)
    inertia = kmeans.inertia_
    
    k_results.append({
        'k': k,
        'silhouette_score': silhouette_avg,
        'inertia': inertia
    })
    
    print(f"k={k}: silhouette_score={silhouette_avg:.3f}, inertia={inertia:.2f}")

k_df = pd.DataFrame(k_results)
print("\nРезультаты для различных k:")
print(k_df)

In [ ]:
# Визуализация результатов
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# График коэффициента силуэта
axes[0].plot(k_df['k'], k_df['silhouette_score'], 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Количество кластеров (k)')
axes[0].set_ylabel('Коэффициент силуэта')
axes[0].set_title('Зависимость коэффициента силуэта от k')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(k_range)

# График инерции (метод локтя)
axes[1].plot(k_df['k'], k_df['inertia'], 'ro-', linewidth=2, markersize=8)
axes[1].set_xlabel('Количество кластеров (k)')
axes[1].set_ylabel('Инерция')
axes[1].set_title('Метод локтя для определения оптимального k')
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(k_range)

plt.tight_layout()
plt.show()

# Определение оптимального k
best_k = k_df.loc[k_df['silhouette_score'].idxmax(), 'k']
best_silhouette = k_df['silhouette_score'].max()

print(f"\nОптимальное количество кластеров: k={best_k}")
print(f"Лучший коэффициент силуэта: {best_silhouette:.3f}")

# Анализ локтя
inertia_diff = k_df['inertia'].diff().diff()
elbow_k = k_df.loc[inertia_diff.idxmax(), 'k']
print(f"\nРекомендуемое k по методу локтя: k={elbow_k}")

In [ ]:
# Визуализация кластеров для оптимального k
kmeans_optimal = KMeans(n_clusters=best_k, random_state=42, n_init=10)
clusters_optimal = kmeans_optimal.fit_predict(X_scaled)

# Используем PCA для 2D визуализации
pca_viz = PCA(n_components=2)
X_pca_viz = pca_viz.fit_transform(X_scaled)

plt.figure(figsize=(12, 5))

# Истинные классы
plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca_viz[:, 0], X_pca_viz[:, 1], c=y, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.title('Истинные классы')
plt.xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.2%} дисперсии)')
plt.ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.2%} дисперсии)')

# Оптимальные кластеры
plt.subplot(1, 2, 2)
scatter = plt.scatter(X_pca_viz[:, 0], X_pca_viz[:, 1], c=clusters_optimal, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.title(f'Оптимальные кластеры (k={best_k})')
plt.xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.2%} дисперсии)')
plt.ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.2%} дисперсии)')

plt.tight_layout()
plt.show()

print(f"Коэффициент силуэта для оптимального k={best_k}: {best_silhouette:.3f}")

## Заключение и выводы

In [ ]:
# Сводная таблица результатов
print("СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:")
print("=" * 50)

print(f"\n1. Исходные данные:")
print(f"   - Количество образцов: {X_scaled.shape[0]}")
print(f"   - Количество признаков: {X_scaled.shape[1]}")
print(f"   - Количество истинных классов: {y.nunique()}")

print(f"\n2. K-Means с k=3:")
print(f"   - Коэффициент силуэта: {silhouette_avg:.3f}")

print(f"\n3. PCA анализ:")
print(f"   - Лучший результат: {pca_df['silhouette_score'].max():.3f} (n_components={best_pca_n})")
print(f"   - Объясненная дисперсия для 2 компонентов: {pca_best.explained_variance_ratio_.sum():.3f}")

print(f"\n4. t-SNE анализ:")
print(f"   - Лучший результат: {tsne_df['silhouette_score'].max():.3f} (n_components={best_tsne_n})")

print(f"\n5. Инициализация центроидов:")
kmeans_plus_plus = init_df[init_df['init_method'] == 'k-means++']['silhouette_score']
random_init = init_df[init_df['init_method'] == 'random']['silhouette_score']
print(f"   - k-means++: mean={kmeans_plus_plus.mean():.3f} ± {kmeans_plus_plus.std():.3f}")
print(f"   - random: mean={random_init.mean():.3f} ± {random_init.std():.3f}")

print(f"\n6. Оптимальное количество кластеров:")
print(f"   - По коэффициенту силуэта: k={best_k} (score={best_silhouette:.3f})")
print(f"   - По методу локтя: k={elbow_k}")

print(f"\n7. Сравнение методов:")
print(f"   - Исходные данные: {silhouette_avg:.3f}")
print(f"   - PCA (2 компонента): {silhouette_pca:.3f}")
print(f"   - t-SNE (2 компонента): {silhouette_tsne:.3f}")
print(f"   - Оптимальное k: {best_silhouette:.3f}")

print("\n" + "=" * 50)
print("ВЫВОДЫ:")
print("1. Набор данных содержит 210 образцов зерен пшеницы с 7 числовыми признаками.")
print("2. Стандартизация данных улучшает качество кластеризации.")
print("3. PCA и t-SNE показывают разные результаты в зависимости от количества компонентов.")
print("4. Метод инициализации k-means++ показывает более стабильные результаты.")
print("5. Оптимальное количество кластеров может отличаться от истинного количества классов.")
print("6. Коэффициент силуэта является хорошей метрикой для оценки качества кластеризации.")